In [2]:
#importing necessary library

import pandas as pd
from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')


In [3]:
#database conncection string

db_string = 'mssql+pyodbc://localhost/AdventureWorks2022?Trusted_Connection=yes&Driver=ODBC Driver 17 for SQL Server'

#create db engine

engine = create_engine(db_string)

In [4]:
#1 Retrieve the names of all products in the Product table

data = pd.read_sql_table(schema = 'Production', table_name = 'Product', con=engine)


In [5]:
data['Name']

0                            Adjustable Race
1                               Bearing Ball
2                            BB Ball Bearing
3                      Headset Ball Bearings
4                                      Blade
5                                LL Crankarm
6                                ML Crankarm
7                                HL Crankarm
8                            Chainring Bolts
9                              Chainring Nut
10                                 Chainring
11                                Crown Race
12                               Chain Stays
13                                   Decal 1
14                                   Decal 2
15                                 Down Tube
16                         Mountain End Caps
17                             Road End Caps
18                          Touring End Caps
19                                  Fork End
20                                 Freewheel
21                             Flat Washer 1
22        

In [6]:
#2 Display the addresses of all customers from the Customer table
#method 1 

columns = [
    "AddressID", 
    "AddressLine1", 
    "AddressLine2", 
    "City", 
    "StateProvinceID", 
    "PostalCode", 
    "rowguid", 
    "ModifiedDate"
]

address = pd.read_sql_table(schema = 'Person', table_name = 'Address', con = engine, columns = columns)
address.head() #not recomended as it removes tables

,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,rowguid,ModifiedDate
0,1,1970 Napa Ct.,None,Bothell,79,98011,9aadcb0d-36cf-483f-84d8-585c2d4ec6e9,2007-12-04
1,2,9833 Mt. Dias Blv.,None,Bothell,79,98011,32a54b9e-e034-4bfb-b573-a71cde60d8c0,2008-11-30
2,3,7484 Roundtree Drive,None,Bothell,79,98011,4c506923-6d1b-452c-a07c-baa6f5b142a4,2013-03-07
3,4,9539 Glenside Dr,None,Bothell,79,98011,e5946c78-4bcc-477f-9fa1-cc09de16a880,2009-02-03
4,5,1226 Shoe St.,None,Bothell,79,98011,fbaff937-4a97-4af0-81fd-b849900e9bb0,2008-12-19


In [7]:
#method 2 
#The STAsText() method is specifically designed to work with columns of type geography or geometry

query = """
SELECT 
    AddressID, 
    AddressLine1, 
    AddressLine2, 
    City, 
    StateProvinceID, 
    PostalCode, 
    SpatialLocation.STAsText() AS SpatialLocation, 
    rowguid, 
    ModifiedDate
FROM 
    Person.Address
"""

# Read the data into a pandas DataFrame
address = pd.read_sql_query(query, con=engine)
address.head() #but it will change the data to human readable format

,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,SpatialLocation,rowguid,ModifiedDate
0,1,1970 Napa Ct.,None,Bothell,79,98011,POINT (-122.164644615406 47.7869921906598),9AADCB0D-36CF-483F-84D8-585C2D4EC6E9,2007-12-04
1,2,9833 Mt. Dias Blv.,None,Bothell,79,98011,POINT (-122.250185528911 47.6867097047995),32A54B9E-E034-4BFB-B573-A71CDE60D8C0,2008-11-30
2,3,7484 Roundtree Drive,None,Bothell,79,98011,POINT (-122.274625789912 47.7631154083121),4C506923-6D1B-452C-A07C-BAA6F5B142A4,2013-03-07
3,4,9539 Glenside Dr,None,Bothell,79,98011,POINT (-122.335726442416 47.7392386259644),E5946C78-4BCC-477F-9FA1-CC09DE16A880,2009-02-03
4,5,1226 Shoe St.,None,Bothell,79,98011,POINT (-122.091323832402 47.7010357742081),FBAFF937-4A97-4AF0-81FD-B849900E9BB0,2008-12-19


In [8]:
#method3

query = """
SELECT 
    AddressID, 
    AddressLine1, 
    AddressLine2, 
    City, 
    StateProvinceID, 
    PostalCode, 
    CONVERT(varbinary(max), SpatialLocation) AS SpatialLocation, 
    rowguid, 
    ModifiedDate
FROM 
    Person.Address
"""


address = pd.read_sql_query(query, con=engine)

# Convert the binary data to a hexadecimal string representation
address['SpatialLocation'] = address['SpatialLocation'].apply(lambda x: x.hex() if x else None)


address.head()

,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,SpatialLocation,rowguid,ModifiedDate
0,1,1970 Napa Ct.,None,Bothell,79,98011,e6100000010cae8bfc28bce4474067a89189898a5ec0,9AADCB0D-36CF-483F-84D8-585C2D4EC6E9,2007-12-04
1,2,9833 Mt. Dias Blv.,None,Bothell,79,98011,e6100000010cd6fa851ae6d74740bc262a0a03905ec0,32A54B9E-E034-4BFB-B573-A71CDE60D8C0,2008-11-30
2,3,7484 Roundtree Drive,None,Bothell,79,98011,e6100000010c18e304c4ade14740da930c7893915ec0,4C506923-6D1B-452C-A07C-BAA6F5B142A4,2013-03-07
3,4,9539 Glenside Dr,None,Bothell,79,98011,e6100000010c813a0d5f9fde474011a5c28a7c955ec0,E5946C78-4BCC-477F-9FA1-CC09DE16A880,2009-02-03
4,5,1226 Shoe St.,None,Bothell,79,98011,e6100000010c61c64d8abbd94740c460ea3fd8855ec0,FBAFF937-4A97-4AF0-81FD-B849900E9BB0,2008-12-19


In [9]:
address_id = pd.read_sql_table (schema = 'Person', table_name = 'BusinessEntityAddress', con = engine)
address_id.head()

,BusinessEntityID,AddressID,AddressTypeID,rowguid,ModifiedDate
0,2,293,2,84ae7057-edf4-4c51-8b8d-3aeaefbfb4a1,2014-09-12 11:15:06.967
1,3,224,2,3c915b31-7c05-4a05-9859-0df663677240,2014-09-12 11:15:06.967
2,4,11387,2,3dc70cc4-3ae8-424f-8b1f-481c5478e941,2014-09-12 11:15:06.967
3,5,190,2,c0ed2f68-937b-4594-9459-581ac53c98e3,2014-09-12 11:15:06.967
4,6,286,2,4ca1686a-a7df-4bd8-9d7d-82a63210208a,2014-09-12 11:15:06.967


In [10]:
id = pd.read_sql_table (schema = 'Sales', table_name = 'Customer', con = engine)
id.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [11]:
address_1 = pd.merge(address_id, id, how = 'inner', left_on = 'BusinessEntityID', right_on = 'PersonID')
address_1.head()

,BusinessEntityID,AddressID,AddressTypeID,rowguid_x,ModifiedDate_x,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid_y,ModifiedDate_y
0,1699,29177,2,fc261296-1b15-4567-a6e9-73a63754021e,2013-07-31,11377,1699.0,NaN,8,AW00011377,ef5d2cdb-aef9-4480-b860-105b4850b83b,2014-09-12 11:15:07.263
1,1700,16400,2,a6d34b66-1d03-4380-8374-99303cd2b647,2014-05-05,11913,1700.0,NaN,9,AW00011913,dc8a8ada-dfda-43b5-886d-9df12f7f0fa8,2014-09-12 11:15:07.263
2,1701,19867,2,3839aa27-eb1c-410e-bd2a-9421b9df1849,2012-03-06,11952,1701.0,NaN,9,AW00011952,d5730192-f877-41be-9f29-3206991c454f,2014-09-12 11:15:07.263
3,1702,17009,2,422b0bda-1735-4b13-8d3f-86ad9545dd89,2011-09-26,20164,1702.0,NaN,10,AW00020164,c831b5c1-c96d-479e-a416-0525f7239b5a,2014-09-12 11:15:07.263
4,1703,13003,2,f70f230c-9653-4b0d-a930-1890d0515409,2012-03-17,20211,1703.0,NaN,9,AW00020211,1a02e88c-cbbf-4a6f-9a0b-89fb99e57b0d,2014-09-12 11:15:07.263


In [12]:
address_1 = address_1[['PersonID', 'AddressID']]
address_1.head()

,PersonID,AddressID
0,1699.0,29177
1,1700.0,16400
2,1701.0,19867
3,1702.0,17009
4,1703.0,13003


In [13]:
address_final = pd.merge(address_1, address, how = 'inner', on = 'AddressID')
address_final.head()

,PersonID,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,SpatialLocation,rowguid,ModifiedDate
0,1699.0,29177,Pappelallee 6667,None,Solingen,53,42651,e6100000010c5466a392cc99494032efe9df7b711c40,26A8CC05-1299-4A87-946E-D1EBF8D44064,2013-07-31
1,1700.0,16400,1861 Chinquapin Ct,None,Seaford,77,3198,e6100000010c600a0a7bd80743c0317e6cc586246240,728D6E3C-000C-4D33-B7E4-8DADC4A358AC,2014-05-05
2,1701.0,19867,4693 Mills Dr.,None,Geelong,77,3220,e6100000010ca534ac5bbe1243c0553f6dad73066240,9C733C8E-C5E1-457F-9D48-825B1CED4FA6,2012-03-06
3,1702.0,17009,1312 Skycrest Drive,None,Lancaster,14,LA1 1LN,e6100000010c10f2be2e290f4b409cc05d65146d06c0,06E17460-3742-4900-B00F-B0338ACAD068,2011-09-26
4,1703.0,13003,9860 Brookview Drive,None,East Brisbane,64,4169,e6100000010c2959b127ef7a3bc007e9f2355e1f6340,5395B838-14DC-4E3E-89C3-83089F8543A1,2012-03-17


In [14]:
address_final = address_final.iloc[:, [0,1,2,4,5,6]]
address_final.head()

,PersonID,AddressID,AddressLine1,City,StateProvinceID,PostalCode
0,1699.0,29177,Pappelallee 6667,Solingen,53,42651
1,1700.0,16400,1861 Chinquapin Ct,Seaford,77,3198
2,1701.0,19867,4693 Mills Dr.,Geelong,77,3220
3,1702.0,17009,1312 Skycrest Drive,Lancaster,14,LA1 1LN
4,1703.0,13003,9860 Brookview Drive,East Brisbane,64,4169


In [15]:
#3 List the names and salaries of all employees from the Employee table

d1 = pd.read_sql_table(schema = 'Person', table_name = 'Person', con = engine)
d1.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f3a3f6b4-ae3b-430c-a754-9f2231ba6fef,2007-12-30


In [16]:
d2 = pd.read_sql_table(schema = 'HumanResources', table_name = 'EmployeePayHistory', con = engine)
d2.head()

,BusinessEntityID,RateChangeDate,Rate,PayFrequency,ModifiedDate
0,1,2009-01-14,120.0000,2,2014-06-30
1,2,2008-01-31,63.4615,2,2014-06-30
2,3,2007-11-11,43.2692,2,2014-06-30
3,4,2007-12-05,8.6200,2,2007-11-21
4,4,2010-05-31,23.7200,2,2010-05-16


In [17]:
salary = pd.merge(d1, d2, on = 'BusinessEntityID', how = 'inner')
salary.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate_x,RateChangeDate,Rate,PayFrequency,ModifiedDate_y
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07,2009-01-14,120.0000,2,2014-06-30
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24,2008-01-31,63.4615,2,2014-06-30
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04,2007-11-11,43.2692,2,2014-06-30
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28,2007-12-05,8.6200,2,2007-11-21
4,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28,2010-05-31,23.7200,2,2010-05-16


In [18]:
salary = salary.iloc[:, [0,1,4,5,6]]
salary.head()

,BusinessEntityID,PersonType,FirstName,MiddleName,LastName
0,1,EM,Ken,J,Sánchez
1,2,EM,Terri,Lee,Duffy
2,3,EM,Roberto,None,Tamburello
3,4,EM,Rob,None,Walters
4,4,EM,Rob,None,Walters


In [19]:
#4 Show the order IDs and total amounts from the SalesOrderHeader table

due = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con =engine)
due.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [20]:
due = due[['SalesOrderID', 'TotalDue']]
due.head()

,SalesOrderID,TotalDue
0,43660,1457.3288
1,43661,36865.8012
2,43662,32474.9324
3,43663,472.3108
4,43664,27510.4109


In [21]:
#5 Fetch the categories of all products from the ProductCategory table

category = pd.read_sql_table(schema = 'Production', table_name = 'ProductCategory', con = engine)
category.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30


In [22]:
category = category[['ProductCategoryID', 'Name']]
category.head()

,ProductCategoryID,Name
0,1,Bikes
1,2,Components
2,3,Clothing
3,4,Accessories


In [23]:
#6 Retrieve all columns from the Product table

product = pd.read_sql_table(schema = 'Production', table_name = 'Product', con = engine)
product.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [24]:
#7 Display all columns from the SalesOrderHeader table

soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [25]:
#8 List all columns from the Customer table

cus = pd.read_sql_table(schema = 'Sales', table_name = 'Customer', con = engine)
cus.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [26]:
#9 Fetch all columns from the Employee table
# Convert hierarchyid to string

query = '''
SELECT 
    BusinessEntityID,
    NationalIDNumber,
    LoginID,
    CONVERT(varchar(max), OrganizationNode) AS OrganizationNode, 
    OrganizationLevel,
    JobTitle,
    BirthDate,
    MaritalStatus,
    Gender,
    HireDate,
    SalariedFlag,
    VacationHours,
    SickLeaveHours,
    CurrentFlag,
    rowguid,
    ModifiedDate
FROM 
    HumanResources.Employee'''
emp = pd.read_sql_query(query, con = engine)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,/1/,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,/1/1/,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,/1/1/1/,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,/1/1/2/,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [27]:
#10 Display all columns from the ProductCategory table

category = pd.read_sql_table(schema = 'Production', table_name = 'ProductCategory', con = engine)
category.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30


In [28]:
#11 Insert a new product into the Product table

product = pd.read_sql_table(schema = 'Production', table_name = 'Product', con = engine)
product.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [29]:
data = pd.DataFrame({
    'ProductID': [10001],
    'Name': ['nothing'],
    'ProductNumber': ['AX1001'],
    'MakeFlag': [1],
    'FinishedGoodsFlag': [False],
    'Color': [None],
    'SafetyStockLevel': [10],
    'ReorderPoint': [0],
    'StandardCost': [0.0],
    'ListPrice': [0.0],
    'Size': [None],
    'SizeUnitMeasureCode': [None],
    'WeightUnitMeasureCode': [None],
    'Weight': [None],
    'DaysToManufacture': [0],
    'ProductLine': [None],
    'Class': [None],
    'Style': [None],
    'ProductSubcategoryID': [None],
    'ProductModelID': [None],
    'SellStartDate': [pd.to_datetime('2023-01-01')],
    'SellEndDate': [None],
    'DiscontinuedDate': [None],
    'rowguid': ['some-unique-identifier'],
    'ModifiedDate': [pd.to_datetime('2023-01-01')]
})

In [30]:
pd.concat([product,data], axis = 0).tail()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
502,998,"Road-750 Black, 48",BK-R19B-48,1,True,Black,100,75,343.6496,146.674,48,CM,LB,20.13,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,3de9a212-1d49-40b6-b10a-f564d981dbde,2014-02-08 10:01:36.827
503,999,"Road-750 Black, 52",BK-R19B-52,1,True,Black,100,75,343.6496,146.674,52,CM,LB,20.42,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,ae638923-2b67-4679-b90e-abbab17dca31,2014-02-08 10:01:36.827
504,1001,Vintage Handcrafted Wooden Handlebars,VHWH1000,1,True,GOLD,3,4,6.9223,8.900,M,CM,LB,1.50,1,M,H,U,23.0,18.0,2011-05-31 10:00:00,2011-08-31 10:00:00,2012-05-31 10:00:00,4fbd47f2-b7e5-48b0-928e-45b8933a56da,2014-02-08 10:02:36.827
505,2006,BIKE HANDLE,RV-H123,1,True,BLUE,250,20,145.8800,2000.000,20,CM,LB,2.00,0,None,None,None,NaN,NaN,2008-04-30 00:00:00,NaT,NaT,d67538f2-a81a-4a4f-98f7-202002d888b4,2024-03-02 14:57:55.763
0,10001,nothing,AX1001,1,False,None,10,0,0.0000,0.000,None,None,None,NaN,0,None,None,None,NaN,NaN,2023-01-01 00:00:00,NaT,NaT,some-unique-identifier,2023-01-01 00:00:00.000


In [31]:
# 12 Add a new customer into the Customer table

customer = pd.read_sql_table(schema = 'Sales', table_name = 'Customer', con = engine)
customer.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [32]:
new = pd.DataFrame({'CustomerID'  : [100011], 'PersonID' : [2035]})
new

,CustomerID,PersonID
0,100011,2035


In [33]:
pd.concat([customer, new]).head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1.0,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4.0,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4.0,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4.0,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4.0,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [34]:
#13 Insert a new employee into the Employee table

query = '''
SELECT 
    BusinessEntityID,
    NationalIDNumber,
    LoginID,
    CONVERT(varchar(max), OrganizationNode) AS OrganizationNode, 
    OrganizationLevel,
    JobTitle,
    BirthDate,
    MaritalStatus,
    Gender,
    HireDate,
    SalariedFlag,
    VacationHours,
    SickLeaveHours,
    CurrentFlag,
    rowguid,
    ModifiedDate
FROM 
    HumanResources.Employee'''
emp = pd.read_sql_query(query, con = engine)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,/1/,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,/1/1/,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,/1/1/1/,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,/1/1/2/,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [35]:
new_emp = pd.DataFrame({'BusinessEntityID' : [2034444], 'NationalIDNumber' : [2034444444], 'JobTitle' : ['Intern'], 'MaritalStatus' : ['S'], 'Gender' : ['M']})
new_emp

,BusinessEntityID,NationalIDNumber,JobTitle,MaritalStatus,Gender
0,2034444,2034444444,Intern,S,M


In [36]:
pd.concat([emp, new_emp]).tail()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
287,288,954276278,adventure-works\rachel0,/6/3/1/,3.0,Sales Representative,1975-07-09,S,F,2013-05-30,True,35.0,37.0,True,B9BF7741-E0CA-4F37-ACDE-A4F78C6D03E9,2014-06-30
288,289,668991357,adventure-works\jae0,/6/3/2/,3.0,Sales Representative,1968-03-17,M,F,2012-05-30,True,37.0,38.0,True,723A5921-D8A1-4659-9BC4-13C4CF7C9C91,2014-06-30
289,290,134219713,adventure-works\ranjit0,/6/3/3/,3.0,Sales Representative,1975-09-30,S,M,2012-05-30,True,34.0,37.0,True,604213F9-DD0F-43B4-BDD2-C96E93D3F4BF,2014-06-30
290,291,10000001,adventure-works\ken12,None,NaN,Design Engineer,1952-09-27,S,M,2008-01-24,False,5.0,23.0,True,8C5B9C80-0701-4629-9C5E-BD49743A3DF2,2014-06-30
0,2034444,2034444444,NaN,NaN,NaN,Intern,NaN,S,M,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [37]:
# 14 Add a new sales order into the SalesOrderHeader table

soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [38]:
S = pd.DataFrame({'SalesOrderID' : [6000001], 'OrderDate' : ['21/03/2024']})
S

,SalesOrderID,OrderDate
0,6000001,21/03/2024


In [39]:
pd.concat([soh, S]).tail()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
31461,75121,8.0,2014-06-30 00:00:00,2014-07-12,2014-07-07,5.0,True,SO75121,None,10-4030-015251,15251.0,NaN,6.0,26553.0,26553.0,1.0,14220.0,529864Vi73738,NaN,74.9800,5.9984,1.8745,82.8529,None,d7395c0e-00cb-4bfa-a238-0d6a9f49884f,2014-07-07
31462,75122,8.0,2014-06-30 00:00:00,2014-07-12,2014-07-07,5.0,True,SO75122,None,10-4030-015868,15868.0,NaN,6.0,14616.0,14616.0,1.0,18719.0,330022Vi97312,NaN,30.9700,2.4776,0.7743,34.2219,None,4221035a-4159-492f-af40-4363a64ffc16,2014-07-07
31463,75123,8.0,2014-06-30 00:00:00,2014-07-12,2014-07-07,5.0,True,SO75123,None,10-4030-018759,18759.0,NaN,6.0,14024.0,14024.0,1.0,10084.0,230370Vi51970,NaN,189.9700,15.1976,4.7493,209.9169,None,d54752ff-2b54-4be5-95ea-3b72289c059f,2014-07-07
31464,75124,8.0,2011-05-31 00:00:00,2011-06-12,2011-06-07,5.0,False,SO75124,PO18009186470,-4656,29614.0,283.0,5.0,517.0,849.0,5.0,15232.0,55680Vi53503,4.0,20565.6206,1971.5149,616.0984,23153.2339,None,0f91dbcb-04ad-4d4d-aeed-c2af783d81b9,2011-06-07
0,6000001,NaN,21/03/2024,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [40]:
# 15 Insert a new product category into the ProductCategory table

cat = pd.read_sql_table(schema = 'Production', table_name = 'ProductCategory', con = engine)
cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30


In [41]:
new = pd.DataFrame({'ProductCategoryID' : [5], 'Name' : ['nothing'], 'ModifiedDate' : ['2024-05-30']})
new

,ProductCategoryID,Name,ModifiedDate
0,5,nothing,2024-05-30


In [42]:
pd.concat([cat, new]).tail()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30 00:00:00
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30 00:00:00
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30 00:00:00
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30 00:00:00
0,5,nothing,NaN,2024-05-30


In [43]:
# 16 Update the price of a specific product in the Product table

prod = pd.read_sql_table(schema = 'Production', table_name = 'Product', con =engine)
prod.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [44]:
prod.loc[0, ['Name']] = 'Nothing'
prod.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Nothing,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [45]:
#17 Update the address of a particular customer in the Customer table

#bill to address table
soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [46]:
#customer table
customer = pd.read_sql_table(schema = 'Sales', table_name = 'Customer', con = engine)
customer.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [47]:
#address table
columns = [
    "AddressID", 
    "AddressLine1", 
    "AddressLine2", 
    "City", 
    "StateProvinceID", 
    "PostalCode", 
    "rowguid", 
    "ModifiedDate"
]

address = pd.read_sql_table(schema = 'Person', table_name = 'Address', con = engine, columns = columns)
address.head()

,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,rowguid,ModifiedDate
0,1,1970 Napa Ct.,None,Bothell,79,98011,9aadcb0d-36cf-483f-84d8-585c2d4ec6e9,2007-12-04
1,2,9833 Mt. Dias Blv.,None,Bothell,79,98011,32a54b9e-e034-4bfb-b573-a71cde60d8c0,2008-11-30
2,3,7484 Roundtree Drive,None,Bothell,79,98011,4c506923-6d1b-452c-a07c-baa6f5b142a4,2013-03-07
3,4,9539 Glenside Dr,None,Bothell,79,98011,e5946c78-4bcc-477f-9fa1-cc09de16a880,2009-02-03
4,5,1226 Shoe St.,None,Bothell,79,98011,fbaff937-4a97-4af0-81fd-b849900e9bb0,2008-12-19


In [48]:
add_id = pd.merge(soh, customer, how = 'inner', left_on = 'CustomerID', right_on = 'PersonID')
add_id.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber_x,CustomerID_x,SalesPersonID,TerritoryID_x,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid_x,ModifiedDate_x,CustomerID_y,PersonID,StoreID,TerritoryID_y,AccountNumber_y,rowguid_y,ModifiedDate_y
0,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,14501,NaN,4,20278,20278,1,12860.0,733791Vi66509,NaN,699.0982,55.9279,17.4775,772.5036,None,5509a5b0-a6f3-4209-9728-148fb2d15d59,2011-06-07,20650,14501.0,NaN,4,AW00020650,24b676fc-3ecb-4928-a6ac-a46f01ad900a,2014-09-12 11:15:07.263
1,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,11003,NaN,9,20425,20425,1,11150.0,335093Vi57504,2.0,3399.9900,271.9992,84.9998,3756.9890,None,4b3a130d-02ae-47aa-a91e-726b39440613,2011-06-07,16365,11003.0,NaN,6,AW00016365,d033118c-1992-4033-b791-3a7cf6ee3efc,2014-09-12 11:15:07.263
2,43703,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43703,None,10-4030-016624,16624,NaN,9,23967,23967,1,9497.0,1234632Vi48985,15.0,3578.2700,286.2616,89.4568,3953.9884,None,cc630723-57f1-4e59-b1ef-bf77930b3906,2011-06-08,17277,16624.0,NaN,9,AW00017277,6495ab89-ea7d-4d57-a077-60d8db1579ee,2014-09-12 11:15:07.263
3,43704,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43704,None,10-4030-011005,11005,NaN,9,16025,16025,1,1988.0,835095Vi10342,15.0,3374.9900,269.9992,84.3748,3729.3640,None,b09af2d1-243d-444f-9784-fcfceb3fbb0a,2011-06-08,29300,11005.0,NaN,9,AW00029300,6fbb1c53-1734-4838-9045-0d50f35765d8,2014-09-12 11:15:07.263
4,43705,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43705,None,10-4030-011011,11011,NaN,9,27345,27345,1,7205.0,1035101Vi37381,15.0,3399.9900,271.9992,84.9998,3756.9890,None,17a0ede6-6f1a-419d-bcf9-3638b79166d0,2011-06-08,11916,11011.0,NaN,9,AW00011916,9451cd04-8197-489e-98cd-67a2e1f5d5b5,2014-09-12 11:15:07.263


In [49]:
add = pd.merge(add_id, address, how = 'inner', right_on = 'AddressID', left_on = 'BillToAddressID')
add.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber_x,CustomerID_x,SalesPersonID,TerritoryID_x,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid_x,ModifiedDate_x,CustomerID_y,PersonID,StoreID,TerritoryID_y,AccountNumber_y,rowguid_y,ModifiedDate_y,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,rowguid,ModifiedDate
0,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,14501,NaN,4,20278,20278,1,12860.0,733791Vi66509,NaN,699.0982,55.9279,17.4775,772.5036,None,5509a5b0-a6f3-4209-9728-148fb2d15d59,2011-06-07,20650,14501.0,NaN,4,AW00020650,24b676fc-3ecb-4928-a6ac-a46f01ad900a,2014-09-12 11:15:07.263,20278,249 Alexander Pl.,None,Beverly Hills,9,90210,ee1948e9-bc9a-4815-8124-34c470a1e4c8,2011-05-31
1,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,11003,NaN,9,20425,20425,1,11150.0,335093Vi57504,2.0,3399.9900,271.9992,84.9998,3756.9890,None,4b3a130d-02ae-47aa-a91e-726b39440613,2011-06-07,16365,11003.0,NaN,6,AW00016365,d033118c-1992-4033-b791-3a7cf6ee3efc,2014-09-12 11:15:07.263,20425,1825 Village Pl.,None,North Ryde,50,2113,256798c4-c565-40d8-8f98-bd43fa2d080f,2011-05-31
2,43703,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43703,None,10-4030-016624,16624,NaN,9,23967,23967,1,9497.0,1234632Vi48985,15.0,3578.2700,286.2616,89.4568,3953.9884,None,cc630723-57f1-4e59-b1ef-bf77930b3906,2011-06-08,17277,16624.0,NaN,9,AW00017277,6495ab89-ea7d-4d57-a077-60d8db1579ee,2014-09-12 11:15:07.263,23967,3541 Corte Poquito,None,Perth,66,6006,106e4c9d-8eb5-4f75-9c6c-81c3be3b3963,2011-06-01
3,43704,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43704,None,10-4030-011005,11005,NaN,9,16025,16025,1,1988.0,835095Vi10342,15.0,3374.9900,269.9992,84.3748,3729.3640,None,b09af2d1-243d-444f-9784-fcfceb3fbb0a,2011-06-08,29300,11005.0,NaN,9,AW00029300,6fbb1c53-1734-4838-9045-0d50f35765d8,2014-09-12 11:15:07.263,16025,7305 Humphrey Drive,None,East Brisbane,64,4169,8e86b107-c1c8-492f-b39c-8e71f44a16ba,2011-06-01
4,43705,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43705,None,10-4030-011011,11011,NaN,9,27345,27345,1,7205.0,1035101Vi37381,15.0,3399.9900,271.9992,84.9998,3756.9890,None,17a0ede6-6f1a-419d-bcf9-3638b79166d0,2011-06-08,11916,11011.0,NaN,9,AW00011916,9451cd04-8197-489e-98cd-67a2e1f5d5b5,2014-09-12 11:15:07.263,27345,1224 Shoenic,None,East Brisbane,64,4169,3f58f3cf-9788-4da8-9d64-0ecf540b6af6,2011-06-01


In [50]:
add[['PersonID', 'AddressID', 'AddressLine1', 'City', 'PostalCode']].head()

,PersonID,AddressID,AddressLine1,City,PostalCode
0,14501.0,20278,249 Alexander Pl.,Beverly Hills,90210
1,11003.0,20425,1825 Village Pl.,North Ryde,2113
2,16624.0,23967,3541 Corte Poquito,Perth,6006
3,11005.0,16025,7305 Humphrey Drive,East Brisbane,4169
4,11011.0,27345,1224 Shoenic,East Brisbane,4169


In [51]:
# 18 Update the salary of an employee in the Employee table

#pay table
d1 = pd.read_sql_table(schema = 'HumanResources', table_name = 'EmployeePayHistory', con = engine)
d1.head()

,BusinessEntityID,RateChangeDate,Rate,PayFrequency,ModifiedDate
0,1,2009-01-14,120.0000,2,2014-06-30
1,2,2008-01-31,63.4615,2,2014-06-30
2,3,2007-11-11,43.2692,2,2014-06-30
3,4,2007-12-05,8.6200,2,2007-11-21
4,4,2010-05-31,23.7200,2,2010-05-16


In [52]:
#employee table
d2 = pd.read_sql_table(schema = 'Person', table_name = 'Person', con = engine)
d2.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f3a3f6b4-ae3b-430c-a754-9f2231ba6fef,2007-12-30


In [53]:
pay = pd.merge(d1, d2, how = 'inner', left_on = 'BusinessEntityID', right_on = 'BusinessEntityID')
pay.head()

,BusinessEntityID,RateChangeDate,Rate,PayFrequency,ModifiedDate_x,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate_y
0,1,2009-01-14,120.0000,2,2014-06-30,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07
1,2,2008-01-31,63.4615,2,2014-06-30,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24
2,3,2007-11-11,43.2692,2,2014-06-30,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04
3,4,2007-12-05,8.6200,2,2007-11-21,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28
4,4,2010-05-31,23.7200,2,2010-05-16,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28


In [54]:
# 19 Update the order status of a sales order in the SalesOrderHeader table

soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [55]:
soh.loc[0, 'Status'] = 6
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,6,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [56]:
#20 Update the description of a product category in the ProductCategory table

des = pd.read_sql_table(schema = 'Production', table_name = 'ProductDescription', con =engine)
des.head()

,ProductDescriptionID,Description,rowguid,ModifiedDate
0,3,NEW DESCRIPTION,301eed3a-1a82-4855-99cb-2afe8290d641,2013-04-30
1,4,Aluminum alloy cups; large diameter spindle.,dfeba528-da11-4650-9d86-cafda7294eb0,2013-04-30
2,5,Aluminum alloy cups and a hollow axle.,f7178da7-1a7e-4997-8470-06737181305e,2013-04-30
3,8,"Suitable for any type of riding, on or off-roa...",8e6746e5-ad97-46e2-bd24-fcea075c3b52,2013-04-30
4,64,This bike delivers a high-level of performance...,7b1c4e90-85e2-4792-b47b-e0c424e2ec94,2013-04-30


In [57]:
des.loc[des['ProductDescriptionID'] == 3, 'Description'] = 'Updated Description'

In [58]:
des.head()

,ProductDescriptionID,Description,rowguid,ModifiedDate
0,3,Updated Description,301eed3a-1a82-4855-99cb-2afe8290d641,2013-04-30
1,4,Aluminum alloy cups; large diameter spindle.,dfeba528-da11-4650-9d86-cafda7294eb0,2013-04-30
2,5,Aluminum alloy cups and a hollow axle.,f7178da7-1a7e-4997-8470-06737181305e,2013-04-30
3,8,"Suitable for any type of riding, on or off-roa...",8e6746e5-ad97-46e2-bd24-fcea075c3b52,2013-04-30
4,64,This bike delivers a high-level of performance...,7b1c4e90-85e2-4792-b47b-e0c424e2ec94,2013-04-30


In [59]:
# 21 Delete a specific product from the Product table

prod = pd.read_sql_table(schema = 'Production', table_name = 'Product', con =engine)
prod.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [60]:
prod.drop(0).head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827
5,317,LL Crankarm,CA-5965,False,False,Black,500,375,0.0,0.0,None,None,None,NaN,0,None,L,None,NaN,NaN,2008-04-30,NaT,NaT,3c9d10b7-a6b2-4774-9963-c19dcee72fea,2014-02-08 10:01:36.827


In [61]:
#22 Remove a customer from the Customer table

customer = pd.read_sql_table(schema = 'Sales', table_name = 'Customer', con = engine)
customer.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [62]:
customer.drop(0).head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263
5,7,NaN,930.0,1,AW00000007,03e9273e-b193-448e-9823-fe0c44aeed78,2014-09-12 11:15:07.263


In [63]:
# 23 Delete an employee from the Employee table

query = '''
SELECT 
    BusinessEntityID,
    NationalIDNumber,
    LoginID,
    CONVERT(varchar(max), OrganizationNode) AS OrganizationNode, 
    OrganizationLevel,
    JobTitle,
    BirthDate,
    MaritalStatus,
    Gender,
    HireDate,
    SalariedFlag,
    VacationHours,
    SickLeaveHours,
    CurrentFlag,
    rowguid,
    ModifiedDate
FROM 
    HumanResources.Employee'''
emp = pd.read_sql_query(query, con = engine)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,/1/,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,/1/1/,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,/1/1/1/,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,/1/1/2/,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [64]:
emp.drop(0).head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
1,2,245797967,adventure-works\terri0,/1/,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,/1/1/,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,/1/1/1/,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,/1/1/2/,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30
5,6,998320692,adventure-works\jossef0,/1/1/3/,3.0,Design Engineer,1959-03-11,M,M,2008-01-24,True,6,23,True,E39056F1-9CD5-478D-8945-14ACA7FBDCDD,2014-06-30


In [65]:
# 24 Remove a sales order from the SalesOrderHeader table

sale = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con =engine)
sale.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [66]:
sale.drop(0).head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07
5,43665,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43665,PO16588191572,10-4020-000146,29580,283.0,1,849,849,5,15232.0,35568Vi78804,NaN,14352.7713,1375.9427,429.9821,16158.6961,None,5602c304-853c-43d7-9e79-76e320d476cf,2011-06-07


In [67]:
# 25 Delete a product category from the ProductCategory table

cat = pd.read_sql_table(schema = 'Production', table_name = 'ProductCategory', con = engine)
cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30


In [68]:
a = cat.drop(3)
a

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30


In [69]:
#26 Retrieve the names of all products from the Product table ordered alphabetically

data = pd.read_sql_table(schema = 'Production', table_name = 'Product', con=engine)
data.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [70]:
data.sort_values(by = 'Name').head(25)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
216,712,AWC Logo Cap,CA-1098,False,True,Multi,4,3,6.9223,8.99,None,None,None,NaN,0,S,None,U,19.0,2.0,2011-05-31,NaT,NaT,b9ede243-a6f4-4629-b1d4-ffe1aedc6de7,2014-02-08 10:01:36.827
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,1000.00,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
383,879,All-Purpose Bike Stand,ST-1401,False,True,None,4,3,59.4660,159.00,None,None,None,NaN,0,M,None,None,27.0,122.0,2013-05-30,NaT,NaT,c7bb564b-a637-40f5-b21b-cbf7e4f713be,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0000,0.00,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
505,2006,BIKE HANDLE,RV-H123,True,True,BLUE,250,20,145.8800,2000.00,20,CM,LB,2.0,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,d67538f2-a81a-4a4f-98f7-202002d888b4,2024-03-02 14:57:55.763
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0000,0.00,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
381,877,Bike Wash - Dissolver,CL-9009,False,True,None,4,3,2.9733,7.95,None,None,None,NaN,0,S,None,None,29.0,119.0,2013-05-30,NaT,NaT,3c40b5ad-e328-4715-88a7-ec3220f02acf,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0000,0.00,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827
347,843,Cable Lock,LO-C100,False,True,None,4,3,10.3125,25.00,None,None,None,NaN,0,S,None,None,34.0,115.0,2012-05-30,2013-05-29,NaT,56ffd7b9-1014-4640-b1bd-b2649589b4d7,2014-02-08 10:01:36.827
456,952,Chain,CH-0234,False,True,Silver,500,375,8.9866,20.24,None,None,None,NaN,1,None,None,None,7.0,98.0,2013-05-30,NaT,NaT,5d27e2a5-27ec-4ccb-ba2c-fc980ffe6708,2014-02-08 10:01:36.827


In [71]:
#27 Display the addresses of all customers from the Customer table ordered by postal code

#bill to address table
soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [72]:
#customer table
customer = pd.read_sql_table(schema = 'Sales', table_name = 'Customer', con = engine)
customer.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [73]:
#address table
columns = [
    "AddressID", 
    "AddressLine1", 
    "AddressLine2", 
    "City", 
    "StateProvinceID", 
    "PostalCode", 
    "rowguid", 
    "ModifiedDate"
]

address = pd.read_sql_table(schema = 'Person', table_name = 'Address', con = engine, columns = columns)
address.head()

,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,rowguid,ModifiedDate
0,1,1970 Napa Ct.,None,Bothell,79,98011,9aadcb0d-36cf-483f-84d8-585c2d4ec6e9,2007-12-04
1,2,9833 Mt. Dias Blv.,None,Bothell,79,98011,32a54b9e-e034-4bfb-b573-a71cde60d8c0,2008-11-30
2,3,7484 Roundtree Drive,None,Bothell,79,98011,4c506923-6d1b-452c-a07c-baa6f5b142a4,2013-03-07
3,4,9539 Glenside Dr,None,Bothell,79,98011,e5946c78-4bcc-477f-9fa1-cc09de16a880,2009-02-03
4,5,1226 Shoe St.,None,Bothell,79,98011,fbaff937-4a97-4af0-81fd-b849900e9bb0,2008-12-19


In [74]:
#joining soh and customer table

add_id = pd.merge(soh, customer, how = 'inner', left_on = 'CustomerID', right_on = 'PersonID')
add_id.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber_x,CustomerID_x,SalesPersonID,TerritoryID_x,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid_x,ModifiedDate_x,CustomerID_y,PersonID,StoreID,TerritoryID_y,AccountNumber_y,rowguid_y,ModifiedDate_y
0,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,14501,NaN,4,20278,20278,1,12860.0,733791Vi66509,NaN,699.0982,55.9279,17.4775,772.5036,None,5509a5b0-a6f3-4209-9728-148fb2d15d59,2011-06-07,20650,14501.0,NaN,4,AW00020650,24b676fc-3ecb-4928-a6ac-a46f01ad900a,2014-09-12 11:15:07.263
1,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,11003,NaN,9,20425,20425,1,11150.0,335093Vi57504,2.0,3399.9900,271.9992,84.9998,3756.9890,None,4b3a130d-02ae-47aa-a91e-726b39440613,2011-06-07,16365,11003.0,NaN,6,AW00016365,d033118c-1992-4033-b791-3a7cf6ee3efc,2014-09-12 11:15:07.263
2,43703,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43703,None,10-4030-016624,16624,NaN,9,23967,23967,1,9497.0,1234632Vi48985,15.0,3578.2700,286.2616,89.4568,3953.9884,None,cc630723-57f1-4e59-b1ef-bf77930b3906,2011-06-08,17277,16624.0,NaN,9,AW00017277,6495ab89-ea7d-4d57-a077-60d8db1579ee,2014-09-12 11:15:07.263
3,43704,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43704,None,10-4030-011005,11005,NaN,9,16025,16025,1,1988.0,835095Vi10342,15.0,3374.9900,269.9992,84.3748,3729.3640,None,b09af2d1-243d-444f-9784-fcfceb3fbb0a,2011-06-08,29300,11005.0,NaN,9,AW00029300,6fbb1c53-1734-4838-9045-0d50f35765d8,2014-09-12 11:15:07.263
4,43705,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43705,None,10-4030-011011,11011,NaN,9,27345,27345,1,7205.0,1035101Vi37381,15.0,3399.9900,271.9992,84.9998,3756.9890,None,17a0ede6-6f1a-419d-bcf9-3638b79166d0,2011-06-08,11916,11011.0,NaN,9,AW00011916,9451cd04-8197-489e-98cd-67a2e1f5d5b5,2014-09-12 11:15:07.263


In [75]:
#joining the above joined table and address_id table

add = pd.merge(add_id, address, how = 'inner', right_on = 'AddressID', left_on = 'BillToAddressID')
add.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber_x,CustomerID_x,SalesPersonID,TerritoryID_x,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid_x,ModifiedDate_x,CustomerID_y,PersonID,StoreID,TerritoryID_y,AccountNumber_y,rowguid_y,ModifiedDate_y,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,rowguid,ModifiedDate
0,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,14501,NaN,4,20278,20278,1,12860.0,733791Vi66509,NaN,699.0982,55.9279,17.4775,772.5036,None,5509a5b0-a6f3-4209-9728-148fb2d15d59,2011-06-07,20650,14501.0,NaN,4,AW00020650,24b676fc-3ecb-4928-a6ac-a46f01ad900a,2014-09-12 11:15:07.263,20278,249 Alexander Pl.,None,Beverly Hills,9,90210,ee1948e9-bc9a-4815-8124-34c470a1e4c8,2011-05-31
1,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,11003,NaN,9,20425,20425,1,11150.0,335093Vi57504,2.0,3399.9900,271.9992,84.9998,3756.9890,None,4b3a130d-02ae-47aa-a91e-726b39440613,2011-06-07,16365,11003.0,NaN,6,AW00016365,d033118c-1992-4033-b791-3a7cf6ee3efc,2014-09-12 11:15:07.263,20425,1825 Village Pl.,None,North Ryde,50,2113,256798c4-c565-40d8-8f98-bd43fa2d080f,2011-05-31
2,43703,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43703,None,10-4030-016624,16624,NaN,9,23967,23967,1,9497.0,1234632Vi48985,15.0,3578.2700,286.2616,89.4568,3953.9884,None,cc630723-57f1-4e59-b1ef-bf77930b3906,2011-06-08,17277,16624.0,NaN,9,AW00017277,6495ab89-ea7d-4d57-a077-60d8db1579ee,2014-09-12 11:15:07.263,23967,3541 Corte Poquito,None,Perth,66,6006,106e4c9d-8eb5-4f75-9c6c-81c3be3b3963,2011-06-01
3,43704,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43704,None,10-4030-011005,11005,NaN,9,16025,16025,1,1988.0,835095Vi10342,15.0,3374.9900,269.9992,84.3748,3729.3640,None,b09af2d1-243d-444f-9784-fcfceb3fbb0a,2011-06-08,29300,11005.0,NaN,9,AW00029300,6fbb1c53-1734-4838-9045-0d50f35765d8,2014-09-12 11:15:07.263,16025,7305 Humphrey Drive,None,East Brisbane,64,4169,8e86b107-c1c8-492f-b39c-8e71f44a16ba,2011-06-01
4,43705,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43705,None,10-4030-011011,11011,NaN,9,27345,27345,1,7205.0,1035101Vi37381,15.0,3399.9900,271.9992,84.9998,3756.9890,None,17a0ede6-6f1a-419d-bcf9-3638b79166d0,2011-06-08,11916,11011.0,NaN,9,AW00011916,9451cd04-8197-489e-98cd-67a2e1f5d5b5,2014-09-12 11:15:07.263,27345,1224 Shoenic,None,East Brisbane,64,4169,3f58f3cf-9788-4da8-9d64-0ecf540b6af6,2011-06-01


In [76]:
add[['PersonID', 'AddressID', 'AddressLine1', 'City', 'PostalCode']].head(15).sort_values(by = 'PostalCode')

,PersonID,AddressID,AddressLine1,City,PostalCode
1,11003.0,20425,1825 Village Pl.,North Ryde,2113
7,16517.0,25446,3196 Peachwillow,Wollongong,2500
11,16529.0,23557,3571 N St.,Goulburn,2580
10,16483.0,28213,9276 Blackwood Drive,Melbourne,3000
3,11005.0,16025,7305 Humphrey Drive,East Brisbane,4169
4,11011.0,27345,1224 Shoenic,East Brisbane,4169
6,16351.0,13951,7418 Jumping St.,Rockhampton,4700
8,13513.0,21867,Am Grossen Dern 4284,Werne,59368
2,16624.0,23967,3541 Corte Poquito,Perth,6006
12,13264.0,18584,Heiderplatz 948,Ingolstadt,85049


In [77]:
# 28 List the names and salaries of all employees from the Employee table ordered by salary in descending order

#pay table
pay = pd.read_sql_table(schema = 'HumanResources', table_name = 'EmployeePayHistory', con = engine)
pay.head()

,BusinessEntityID,RateChangeDate,Rate,PayFrequency,ModifiedDate
0,1,2009-01-14,120.0000,2,2014-06-30
1,2,2008-01-31,63.4615,2,2014-06-30
2,3,2007-11-11,43.2692,2,2014-06-30
3,4,2007-12-05,8.6200,2,2007-11-21
4,4,2010-05-31,23.7200,2,2010-05-16


In [78]:
#employee name 

name = pd.read_sql_table(schema = 'Person', table_name = 'Person', con = engine)
name.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f3a3f6b4-ae3b-430c-a754-9f2231ba6fef,2007-12-30


In [79]:
#joining both tables

name_salary = pd.merge(pay, name, how = 'inner', left_on = 'BusinessEntityID', right_on = 'BusinessEntityID')
name_salary.head()

,BusinessEntityID,RateChangeDate,Rate,PayFrequency,ModifiedDate_x,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate_y
0,1,2009-01-14,120.0000,2,2014-06-30,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07
1,2,2008-01-31,63.4615,2,2014-06-30,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24
2,3,2007-11-11,43.2692,2,2014-06-30,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04
3,4,2007-12-05,8.6200,2,2007-11-21,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28
4,4,2010-05-31,23.7200,2,2010-05-16,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28


In [80]:
name_salary[['BusinessEntityID', 'FirstName', 'Rate']].head(20).sort_values(by = 'Rate', ascending = False)

,BusinessEntityID,FirstName,Rate
0,1,Ken,120.0000
1,2,Terri,63.4615
8,7,Dylan,50.4808
2,3,Roberto,43.2692
11,10,Michael,42.4808
9,8,Diane,40.8654
10,9,Gigi,40.8654
19,16,David,37.5000
15,14,Michael,36.0577
7,6,Jossef,32.6923


In [81]:
# 29 Show the order IDs and total amounts from the SalesOrderHeader table ordered by order date

soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [82]:
soh[['SalesOrderID', 'OrderDate', 'TotalDue']].head().sort_values(by = 'OrderDate')

,SalesOrderID,OrderDate,TotalDue
0,43660,2011-05-20,1457.3288
1,43661,2011-05-31,36865.8012
2,43662,2011-05-31,32474.9324
3,43663,2011-05-31,472.3108
4,43664,2011-05-31,27510.4109


In [83]:
#30 Fetch the categories of all products from the ProductCategory table ordered by category ID

cat = pd.read_sql_table(schema = 'Production', table_name = 'ProductCategory', con = engine)
cat.head().sort_values('ProductCategoryID')

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30


In [84]:
# 31 Retrieve the products from the Product table where the price is greater than $100

prod = pd.read_sql_table(schema = 'Production', table_name = 'Product', con = engine)
prod[prod['ListPrice'] > 1000].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
209,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.3100,1431.5,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827
210,706,"HL Road Frame - Red, 58",FR-R92R-58,True,True,Red,500,375,1059.3100,1431.5,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,9540ff17-2712-4c90-a3d1-8ce5568b2462,2014-02-08 10:01:36.827
221,717,"HL Road Frame - Red, 62",FR-R92R-62,True,True,Red,500,375,868.6342,1431.5,62,CM,LB,2.30,1,R,H,U,14.0,6.0,2011-05-31,NaT,NaT,052e4f8b-0a2a-46b2-9f42-10febcfae416,2014-02-08 10:01:36.827
222,718,"HL Road Frame - Red, 44",FR-R92R-44,True,True,Red,500,375,868.6342,1431.5,44,CM,LB,2.12,1,R,H,U,14.0,6.0,2011-05-31,NaT,NaT,a88d3b54-2cae-43f2-8c6e-ea1d97b46a7c,2014-02-08 10:01:36.827
223,719,"HL Road Frame - Red, 48",FR-R92R-48,True,True,Red,500,375,868.6342,1431.5,48,CM,LB,2.16,1,R,H,U,14.0,6.0,2011-05-31,NaT,NaT,07befc9a-7634-402b-b234-d7797733baaf,2014-02-08 10:01:36.827


In [85]:
# 32 Display the customers from the Customer table where the city is 'Seattle'

#bill to address table
soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [86]:
#customer table
customer = pd.read_sql_table(schema = 'Sales', table_name = 'Customer', con = engine)
customer.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [87]:
#address table
columns = [
    "AddressID", 
    "AddressLine1", 
    "AddressLine2", 
    "City", 
    "StateProvinceID", 
    "PostalCode", 
    "rowguid", 
    "ModifiedDate"
]

address = pd.read_sql_table(schema = 'Person', table_name = 'Address', con = engine, columns = columns)
address.head()

,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,rowguid,ModifiedDate
0,1,1970 Napa Ct.,None,Bothell,79,98011,9aadcb0d-36cf-483f-84d8-585c2d4ec6e9,2007-12-04
1,2,9833 Mt. Dias Blv.,None,Bothell,79,98011,32a54b9e-e034-4bfb-b573-a71cde60d8c0,2008-11-30
2,3,7484 Roundtree Drive,None,Bothell,79,98011,4c506923-6d1b-452c-a07c-baa6f5b142a4,2013-03-07
3,4,9539 Glenside Dr,None,Bothell,79,98011,e5946c78-4bcc-477f-9fa1-cc09de16a880,2009-02-03
4,5,1226 Shoe St.,None,Bothell,79,98011,fbaff937-4a97-4af0-81fd-b849900e9bb0,2008-12-19


In [88]:
add_id = pd.merge(soh, customer, how = 'inner', left_on = 'CustomerID', right_on = 'PersonID')
add_id.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber_x,CustomerID_x,SalesPersonID,TerritoryID_x,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid_x,ModifiedDate_x,CustomerID_y,PersonID,StoreID,TerritoryID_y,AccountNumber_y,rowguid_y,ModifiedDate_y
0,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,14501,NaN,4,20278,20278,1,12860.0,733791Vi66509,NaN,699.0982,55.9279,17.4775,772.5036,None,5509a5b0-a6f3-4209-9728-148fb2d15d59,2011-06-07,20650,14501.0,NaN,4,AW00020650,24b676fc-3ecb-4928-a6ac-a46f01ad900a,2014-09-12 11:15:07.263
1,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,11003,NaN,9,20425,20425,1,11150.0,335093Vi57504,2.0,3399.9900,271.9992,84.9998,3756.9890,None,4b3a130d-02ae-47aa-a91e-726b39440613,2011-06-07,16365,11003.0,NaN,6,AW00016365,d033118c-1992-4033-b791-3a7cf6ee3efc,2014-09-12 11:15:07.263
2,43703,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43703,None,10-4030-016624,16624,NaN,9,23967,23967,1,9497.0,1234632Vi48985,15.0,3578.2700,286.2616,89.4568,3953.9884,None,cc630723-57f1-4e59-b1ef-bf77930b3906,2011-06-08,17277,16624.0,NaN,9,AW00017277,6495ab89-ea7d-4d57-a077-60d8db1579ee,2014-09-12 11:15:07.263
3,43704,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43704,None,10-4030-011005,11005,NaN,9,16025,16025,1,1988.0,835095Vi10342,15.0,3374.9900,269.9992,84.3748,3729.3640,None,b09af2d1-243d-444f-9784-fcfceb3fbb0a,2011-06-08,29300,11005.0,NaN,9,AW00029300,6fbb1c53-1734-4838-9045-0d50f35765d8,2014-09-12 11:15:07.263
4,43705,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43705,None,10-4030-011011,11011,NaN,9,27345,27345,1,7205.0,1035101Vi37381,15.0,3399.9900,271.9992,84.9998,3756.9890,None,17a0ede6-6f1a-419d-bcf9-3638b79166d0,2011-06-08,11916,11011.0,NaN,9,AW00011916,9451cd04-8197-489e-98cd-67a2e1f5d5b5,2014-09-12 11:15:07.263


In [89]:
add = pd.merge(add_id, address, how = 'inner', right_on = 'AddressID', left_on = 'BillToAddressID')
add.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber_x,CustomerID_x,SalesPersonID,TerritoryID_x,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid_x,ModifiedDate_x,CustomerID_y,PersonID,StoreID,TerritoryID_y,AccountNumber_y,rowguid_y,ModifiedDate_y,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,rowguid,ModifiedDate
0,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,14501,NaN,4,20278,20278,1,12860.0,733791Vi66509,NaN,699.0982,55.9279,17.4775,772.5036,None,5509a5b0-a6f3-4209-9728-148fb2d15d59,2011-06-07,20650,14501.0,NaN,4,AW00020650,24b676fc-3ecb-4928-a6ac-a46f01ad900a,2014-09-12 11:15:07.263,20278,249 Alexander Pl.,None,Beverly Hills,9,90210,ee1948e9-bc9a-4815-8124-34c470a1e4c8,2011-05-31
1,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,11003,NaN,9,20425,20425,1,11150.0,335093Vi57504,2.0,3399.9900,271.9992,84.9998,3756.9890,None,4b3a130d-02ae-47aa-a91e-726b39440613,2011-06-07,16365,11003.0,NaN,6,AW00016365,d033118c-1992-4033-b791-3a7cf6ee3efc,2014-09-12 11:15:07.263,20425,1825 Village Pl.,None,North Ryde,50,2113,256798c4-c565-40d8-8f98-bd43fa2d080f,2011-05-31
2,43703,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43703,None,10-4030-016624,16624,NaN,9,23967,23967,1,9497.0,1234632Vi48985,15.0,3578.2700,286.2616,89.4568,3953.9884,None,cc630723-57f1-4e59-b1ef-bf77930b3906,2011-06-08,17277,16624.0,NaN,9,AW00017277,6495ab89-ea7d-4d57-a077-60d8db1579ee,2014-09-12 11:15:07.263,23967,3541 Corte Poquito,None,Perth,66,6006,106e4c9d-8eb5-4f75-9c6c-81c3be3b3963,2011-06-01
3,43704,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43704,None,10-4030-011005,11005,NaN,9,16025,16025,1,1988.0,835095Vi10342,15.0,3374.9900,269.9992,84.3748,3729.3640,None,b09af2d1-243d-444f-9784-fcfceb3fbb0a,2011-06-08,29300,11005.0,NaN,9,AW00029300,6fbb1c53-1734-4838-9045-0d50f35765d8,2014-09-12 11:15:07.263,16025,7305 Humphrey Drive,None,East Brisbane,64,4169,8e86b107-c1c8-492f-b39c-8e71f44a16ba,2011-06-01
4,43705,8,2011-06-01,2011-06-13,2011-06-08,5,True,SO43705,None,10-4030-011011,11011,NaN,9,27345,27345,1,7205.0,1035101Vi37381,15.0,3399.9900,271.9992,84.9998,3756.9890,None,17a0ede6-6f1a-419d-bcf9-3638b79166d0,2011-06-08,11916,11011.0,NaN,9,AW00011916,9451cd04-8197-489e-98cd-67a2e1f5d5b5,2014-09-12 11:15:07.263,27345,1224 Shoenic,None,East Brisbane,64,4169,3f58f3cf-9788-4da8-9d64-0ecf540b6af6,2011-06-01


In [90]:
address = add[['PersonID', 'AddressID', 'AddressLine1', 'City', 'PostalCode']]
address[address['City'] == 'Seattle']

,PersonID,AddressID,AddressLine1,City,PostalCode
947,11325.0,14922,6679 Cornelius Dr,Seattle,98104
963,11630.0,12452,418 Alfred Avenue,Seattle,98104
1180,12167.0,24907,3307 Mt. Trinity Ct.,Seattle,98104
1595,13214.0,25944,6385 Mark Twain,Seattle,98104
1687,14025.0,15200,4286 NE 3rd Court,Seattle,98104
1842,13354.0,25262,5486 Woodruff Ln.,Seattle,98104
2880,13911.0,17020,5976 Gumwood,Seattle,98104
3920,11325.0,14922,6679 Cornelius Dr,Seattle,98104
4459,11882.0,24895,2880 Ponderosa Dr.,Seattle,98104
4621,19005.0,12744,9535 Pear Dr.,Seattle,98104


In [91]:
#33 List the employees from the Employee table where the job title is 'Sales Representative'

query = '''
SELECT 
    BusinessEntityID,
    NationalIDNumber,
    LoginID,
    CONVERT(varchar(max), OrganizationNode) AS OrganizationNode, 
    OrganizationLevel,
    JobTitle,
    BirthDate,
    MaritalStatus,
    Gender,
    HireDate,
    SalariedFlag,
    VacationHours,
    SickLeaveHours,
    CurrentFlag,
    rowguid,
    ModifiedDate
FROM 
    HumanResources.Employee'''
emp = pd.read_sql_query(query, con = engine)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,/1/,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,/1/1/,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,/1/1/1/,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,/1/1/2/,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [92]:
person = pd.read_sql_table(schema = 'Person', table_name = 'Person', con = engine)
person.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f3a3f6b4-ae3b-430c-a754-9f2231ba6fef,2007-12-30


In [93]:
person_merg = pd.merge(emp, person, how = 'inner', left_on = 'BusinessEntityID', right_on = 'BusinessEntityID')
person_merg.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid_x,ModifiedDate_x,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid_y,ModifiedDate_y
0,1,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07
1,2,245797967,adventure-works\terri0,/1/,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24
2,3,509647174,adventure-works\roberto0,/1/1/,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04
3,4,112457891,adventure-works\rob0,/1/1/1/,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28
4,5,695256908,adventure-works\gail0,/1/1/2/,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f3a3f6b4-ae3b-430c-a754-9f2231ba6fef,2007-12-30


In [94]:
job = person_merg[['BusinessEntityID', 'FirstName', 'JobTitle']]
job[job['JobTitle'] == 'Sales Representative']

,BusinessEntityID,FirstName,JobTitle
274,275,Michael,Sales Representative
275,276,Linda,Sales Representative
276,277,Jillian,Sales Representative
277,278,Garrett,Sales Representative
278,279,Tsvi,Sales Representative
279,280,Pamela,Sales Representative
280,281,Shu,Sales Representative
281,282,José,Sales Representative
282,283,David,Sales Representative
283,284,Tete,Sales Representative


In [95]:
# 34 Show the sales orders from the SalesOrderHeader table where the total amount is less than $500

soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [96]:
soh[soh['TotalDue'] < 500].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
36,43696,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43696,PO9947131800,10-4020-000603,29849,279.0,5,977,977,5,1402.0,98787Vi7192,NaN,419.4589,40.2681,12.5838,472.3108,None,7c3e3b6d-e76e-470b-a75e-b20c25f86760,2011-06-07
214,43874,8,2011-07-01,2011-07-13,2011-07-08,5,False,SO43874,PO12122162917,10-4020-000022,29569,277.0,3,562,562,5,13917.0,66171Vi72178,NaN,419.4589,40.2681,12.5838,472.3108,None,19b1a1e5-ec5a-4f8e-a25a-3a2a21219fad,2011-07-08
225,43885,8,2011-07-01,2011-07-13,2011-07-08,5,False,SO43885,PO609186449,10-4020-000492,29757,276.0,4,1070,1070,5,13246.0,66522Vi68706,NaN,68.4000,5.9280,1.8525,76.1805,None,02b2dac2-90dc-413a-b148-a9145a459ac8,2011-07-08
227,43887,8,2011-07-01,2011-07-13,2011-07-08,5,False,SO43887,PO1276169981,10-4020-000548,29493,279.0,5,970,970,5,9478.0,106673Vi48861,NaN,419.4589,40.2681,12.5838,472.3108,None,9e78ed24-f8ba-4c3d-ad89-d9a1a2e51d45,2011-07-08


In [97]:
# 35 Fetch the products from the Product table where the product category is 'Accessories'

prod = pd.read_sql_table(schema = 'Production', table_name = 'Product', con = engine)
prod.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [98]:
prod_sub = pd.read_sql_table(schema = 'Production', table_name = 'ProductSubcategory', con = engine)
prod_sub.head()


,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2d364ade-264a-433c-b092-4fcbf3804e01,2008-04-30
1,2,1,Road Bikes,000310c0-bcc8-42c4-b0c3-45ae611af06b,2008-04-30
2,3,1,Touring Bikes,02c5061d-ecdc-4274-b5f1-e91d76bc3f37,2008-04-30
3,4,2,Handlebars,3ef2c725-7135-4c85-9ae6-ae9a3bdd9283,2008-04-30
4,5,2,Bottom Brackets,a9e54089-8a1e-4cf5-8646-e3801f685934,2008-04-30


In [99]:
prod_cat = pd.read_sql_table(schema = 'Production', table_name = 'ProductCategory', con = engine)
prod_cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30


In [100]:
sub_merge = pd.merge(prod, prod_sub, how = 'inner', on = 'ProductSubcategoryID')
sub_merge.head()

,ProductID,Name_x,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid_x,ModifiedDate_x,ProductCategoryID,Name_y,rowguid_y,ModifiedDate_y
0,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.3100,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30
1,706,"HL Road Frame - Red, 58",FR-R92R-58,True,True,Red,500,375,1059.3100,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,9540ff17-2712-4c90-a3d1-8ce5568b2462,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30
2,707,"Sport-100 Helmet, Red",HL-U509-R,False,True,Red,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,2e1ef41a-c08a-4ff6-8ada-bde58b64a712,2014-02-08 10:01:36.827,4,Helmets,f5e07a33-c9e0-439c-b5f3-9f25fb65becc,2008-04-30
3,708,"Sport-100 Helmet, Black",HL-U509,False,True,Black,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,a25a44fb-c2de-4268-958f-110b8d7621e2,2014-02-08 10:01:36.827,4,Helmets,f5e07a33-c9e0-439c-b5f3-9f25fb65becc,2008-04-30
4,709,"Mountain Bike Socks, M",SO-B909-M,False,True,White,4,3,3.3963,9.50,M,None,None,NaN,0,M,None,U,23.0,18.0,2011-05-31,2012-05-29,NaT,18f95f47-1540-4e02-8f1f-cc1bcb6828d0,2014-02-08 10:01:36.827,3,Socks,701019c3-09fe-4949-8386-c6ce686474e5,2008-04-30


In [101]:
cat_merge = pd.merge(sub_merge, prod_cat, how = 'inner', on = 'ProductCategoryID')
cat_merge.head()

,ProductID,Name_x,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid_x,ModifiedDate_x,ProductCategoryID,Name_y,rowguid_y,ModifiedDate_y,Name,rowguid,ModifiedDate
0,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.3100,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
1,706,"HL Road Frame - Red, 58",FR-R92R-58,True,True,Red,500,375,1059.3100,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,9540ff17-2712-4c90-a3d1-8ce5568b2462,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,707,"Sport-100 Helmet, Red",HL-U509-R,False,True,Red,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,2e1ef41a-c08a-4ff6-8ada-bde58b64a712,2014-02-08 10:01:36.827,4,Helmets,f5e07a33-c9e0-439c-b5f3-9f25fb65becc,2008-04-30,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30
3,708,"Sport-100 Helmet, Black",HL-U509,False,True,Black,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,a25a44fb-c2de-4268-958f-110b8d7621e2,2014-02-08 10:01:36.827,4,Helmets,f5e07a33-c9e0-439c-b5f3-9f25fb65becc,2008-04-30,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30
4,709,"Mountain Bike Socks, M",SO-B909-M,False,True,White,4,3,3.3963,9.50,M,None,None,NaN,0,M,None,U,23.0,18.0,2011-05-31,2012-05-29,NaT,18f95f47-1540-4e02-8f1f-cc1bcb6828d0,2014-02-08 10:01:36.827,3,Socks,701019c3-09fe-4949-8386-c6ce686474e5,2008-04-30,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30


In [102]:
all = cat_merge[['ProductCategoryID','ProductSubcategoryID','ProductID', 'Name', 'Name_x']]
all.head()

,ProductCategoryID,ProductSubcategoryID,ProductID,Name,Name_x
0,2,14.0,680,Components,"HL Road Frame - Black, 58"
1,2,14.0,706,Components,"HL Road Frame - Red, 58"
2,4,31.0,707,Accessories,"Sport-100 Helmet, Red"
3,4,31.0,708,Accessories,"Sport-100 Helmet, Black"
4,3,23.0,709,Clothing,"Mountain Bike Socks, M"


In [103]:
all[all['Name'] == 'Accessories'].head(20)

,ProductCategoryID,ProductSubcategoryID,ProductID,Name,Name_x
2,4,31.0,707,Accessories,"Sport-100 Helmet, Red"
3,4,31.0,708,Accessories,"Sport-100 Helmet, Black"
6,4,31.0,711,Accessories,"Sport-100 Helmet, Blue"
137,4,35.0,842,Accessories,"Touring-Panniers, Large"
138,4,34.0,843,Accessories,Cable Lock
139,4,36.0,844,Accessories,Minipump
140,4,36.0,845,Accessories,Mountain Pump
141,4,33.0,846,Accessories,Taillights - Battery-Powered
142,4,33.0,847,Accessories,Headlights - Dual-Beam
143,4,33.0,848,Accessories,Headlights - Weatherproof


In [104]:
# 36 Retrieve the first name, last name, and email of all employees

query = '''
SELECT 
    BusinessEntityID,
    NationalIDNumber,
    LoginID,
    CONVERT(varchar(max), OrganizationNode) AS OrganizationNode, 
    OrganizationLevel,
    JobTitle,
    BirthDate,
    MaritalStatus,
    Gender,
    HireDate,
    SalariedFlag,
    VacationHours,
    SickLeaveHours,
    CurrentFlag,
    rowguid,
    ModifiedDate
FROM 
    HumanResources.Employee'''
emp = pd.read_sql_query(query, con = engine)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,/1/,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,/1/1/,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,/1/1/1/,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,/1/1/2/,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [105]:
#names
name = pd.read_sql_table(schema = 'Person', table_name = 'Person', con = engine)
name.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f3a3f6b4-ae3b-430c-a754-9f2231ba6fef,2007-12-30


In [106]:
#email

email = pd.read_sql_table(schema = 'Person', table_name = 'EmailAddress', con =engine)
email.head()


,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8a1901e4-671b-431a-871c-eadb2942e9ee,2009-01-07
1,2,2,terri0@adventure-works.com,b5ff9efd-72a2-4f87-830b-f338fdd4d162,2008-01-24
2,3,3,roberto0@adventure-works.com,c8a51084-1c03-4c58-a8b3-55854ae7c499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ed1-0031-4b4a-afd2-77487a556b3b,2007-11-28
4,5,5,gail0@adventure-works.com,e76d2ea3-08e5-409c-bbe2-5dd1cdf89a3b,2007-12-30


In [107]:
emp_email = pd.merge(name, email, how = 'inner', on = 'BusinessEntityID')
emp_email.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid_x,ModifiedDate_x,EmailAddressID,EmailAddress,rowguid_y,ModifiedDate_y
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07,1,newmail@gmail.com,8a1901e4-671b-431a-871c-eadb2942e9ee,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24,2,terri0@adventure-works.com,b5ff9efd-72a2-4f87-830b-f338fdd4d162,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04,3,roberto0@adventure-works.com,c8a51084-1c03-4c58-a8b3-55854ae7c499,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28,4,rob0@adventure-works.com,17703ed1-0031-4b4a-afd2-77487a556b3b,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f3a3f6b4-ae3b-430c-a754-9f2231ba6fef,2007-12-30,5,gail0@adventure-works.com,e76d2ea3-08e5-409c-bbe2-5dd1cdf89a3b,2007-12-30


In [108]:
emp_email[['BusinessEntityID', 'FirstName', 'LastName', 'EmailAddress']].head(15)

,BusinessEntityID,FirstName,LastName,EmailAddress
0,1,Ken,Sánchez,newmail@gmail.com
1,2,Terri,Duffy,terri0@adventure-works.com
2,3,Roberto,Tamburello,roberto0@adventure-works.com
3,4,Rob,Walters,rob0@adventure-works.com
4,5,Gail,Erickson,gail0@adventure-works.com
5,6,Jossef,Goldberg,jossef0@adventure-works.com
6,7,Dylan,Miller,dylan0@adventure-works.com
7,8,Diane,Margheim,diane1@adventure-works.com
8,9,Gigi,Matthew,gigi0@adventure-works.com
9,10,Michael,Raheem,michael6@adventure-works.com


In [109]:
#37 Get the product name, list price, and color of all products

prod = pd.read_sql_table(schema = 'Production', table_name = 'Product', con = engine)
prod.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [110]:
prod[['ProductID', 'Name', 'Color', 'ListPrice']].head(10)

,ProductID,Name,Color,ListPrice
0,1,Adjustable Race,None,1000.0
1,2,Bearing Ball,None,0.0
2,3,BB Ball Bearing,None,0.0
3,4,Headset Ball Bearings,None,0.0
4,316,Blade,None,0.0
5,317,LL Crankarm,Black,0.0
6,318,ML Crankarm,Black,0.0
7,319,HL Crankarm,Black,0.0
8,320,Chainring Bolts,Silver,0.0
9,321,Chainring Nut,Silver,0.0


In [111]:
# 38 Find the category name and description for all product categories

#product

prod = pd.read_sql_table(schema = 'Production', table_name = 'Product', con = engine)
prod.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [112]:
#product sub category

prod_sub = pd.read_sql_table(schema = 'Production', table_name = 'ProductSubcategory', con = engine)
prod_sub.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2d364ade-264a-433c-b092-4fcbf3804e01,2008-04-30
1,2,1,Road Bikes,000310c0-bcc8-42c4-b0c3-45ae611af06b,2008-04-30
2,3,1,Touring Bikes,02c5061d-ecdc-4274-b5f1-e91d76bc3f37,2008-04-30
3,4,2,Handlebars,3ef2c725-7135-4c85-9ae6-ae9a3bdd9283,2008-04-30
4,5,2,Bottom Brackets,a9e54089-8a1e-4cf5-8646-e3801f685934,2008-04-30


In [113]:
#merging sub category with product

sub = pd.merge(prod, prod_sub, how = 'inner', on = 'ProductSubcategoryID')
sub.head()

,ProductID,Name_x,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid_x,ModifiedDate_x,ProductCategoryID,Name_y,rowguid_y,ModifiedDate_y
0,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.3100,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30
1,706,"HL Road Frame - Red, 58",FR-R92R-58,True,True,Red,500,375,1059.3100,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,9540ff17-2712-4c90-a3d1-8ce5568b2462,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30
2,707,"Sport-100 Helmet, Red",HL-U509-R,False,True,Red,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,2e1ef41a-c08a-4ff6-8ada-bde58b64a712,2014-02-08 10:01:36.827,4,Helmets,f5e07a33-c9e0-439c-b5f3-9f25fb65becc,2008-04-30
3,708,"Sport-100 Helmet, Black",HL-U509,False,True,Black,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,a25a44fb-c2de-4268-958f-110b8d7621e2,2014-02-08 10:01:36.827,4,Helmets,f5e07a33-c9e0-439c-b5f3-9f25fb65becc,2008-04-30
4,709,"Mountain Bike Socks, M",SO-B909-M,False,True,White,4,3,3.3963,9.50,M,None,None,NaN,0,M,None,U,23.0,18.0,2011-05-31,2012-05-29,NaT,18f95f47-1540-4e02-8f1f-cc1bcb6828d0,2014-02-08 10:01:36.827,3,Socks,701019c3-09fe-4949-8386-c6ce686474e5,2008-04-30


In [114]:
#product category

prod_cat = pd.read_sql_table(schema = 'Production', table_name = 'ProductCategory', con = engine)
prod_cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30


In [115]:
#merging to get category

cat = pd.merge(sub, prod_cat, how = 'inner', on = 'ProductCategoryID')
cat.head()

,ProductID,Name_x,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid_x,ModifiedDate_x,ProductCategoryID,Name_y,rowguid_y,ModifiedDate_y,Name,rowguid,ModifiedDate
0,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.3100,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
1,706,"HL Road Frame - Red, 58",FR-R92R-58,True,True,Red,500,375,1059.3100,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,9540ff17-2712-4c90-a3d1-8ce5568b2462,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,707,"Sport-100 Helmet, Red",HL-U509-R,False,True,Red,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,2e1ef41a-c08a-4ff6-8ada-bde58b64a712,2014-02-08 10:01:36.827,4,Helmets,f5e07a33-c9e0-439c-b5f3-9f25fb65becc,2008-04-30,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30
3,708,"Sport-100 Helmet, Black",HL-U509,False,True,Black,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,a25a44fb-c2de-4268-958f-110b8d7621e2,2014-02-08 10:01:36.827,4,Helmets,f5e07a33-c9e0-439c-b5f3-9f25fb65becc,2008-04-30,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30
4,709,"Mountain Bike Socks, M",SO-B909-M,False,True,White,4,3,3.3963,9.50,M,None,None,NaN,0,M,None,U,23.0,18.0,2011-05-31,2012-05-29,NaT,18f95f47-1540-4e02-8f1f-cc1bcb6828d0,2014-02-08 10:01:36.827,3,Socks,701019c3-09fe-4949-8386-c6ce686474e5,2008-04-30,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30


In [116]:
#description id

desc_id = pd.read_sql_table(schema = 'Production', table_name = 'ProductModelProductDescriptionCulture', con = engine)
desc_id.head()

,ProductModelID,ProductDescriptionID,CultureID,ModifiedDate
0,1,1199,en,2013-04-30
1,1,1467,ar,2013-04-30
2,1,1589,fr,2013-04-30
3,1,1712,th,2013-04-30
4,1,1838,he,2013-04-30


In [117]:
#description

desc = pd.read_sql_table(schema = 'Production', table_name = 'ProductDescription', con = engine)
desc.head()

,ProductDescriptionID,Description,rowguid,ModifiedDate
0,3,NEW DESCRIPTION,301eed3a-1a82-4855-99cb-2afe8290d641,2013-04-30
1,4,Aluminum alloy cups; large diameter spindle.,dfeba528-da11-4650-9d86-cafda7294eb0,2013-04-30
2,5,Aluminum alloy cups and a hollow axle.,f7178da7-1a7e-4997-8470-06737181305e,2013-04-30
3,8,"Suitable for any type of riding, on or off-roa...",8e6746e5-ad97-46e2-bd24-fcea075c3b52,2013-04-30
4,64,This bike delivers a high-level of performance...,7b1c4e90-85e2-4792-b47b-e0c424e2ec94,2013-04-30


In [118]:
#merging description

descr = pd.merge(desc_id, desc, how = 'inner', on = 'ProductDescriptionID')
descr.head()

,ProductModelID,ProductDescriptionID,CultureID,ModifiedDate_x,Description,rowguid,ModifiedDate_y
0,1,1199,en,2013-04-30,"Light-weight, wind-resistant, packs to fit int...",53e49876-bfe9-43b6-9df6-f928390f1a8c,2013-04-30 00:00:00.000
1,1,1467,ar,2013-04-30,علب خفيفة الوزن، ومقاومة للريح، تناسب حجم الجيب.,53c4834c-b061-4439-b408-09e77e846423,2013-04-30 00:00:00.000
2,1,1589,fr,2013-04-30,Sacs légers et résistants au vent ; tiennent d...,760a56e1-e358-4448-859e-9cc755fab8e6,2013-04-30 00:00:00.000
3,1,1712,th,2013-04-30,น้ำหนักเบา กันลม ขนาดกะทัดรัดพอดีกระเป๋า,f7f20598-dad3-43aa-864f-c2c84f1ac616,2014-02-08 10:32:17.973
4,1,1838,he,2013-04-30,"קל-משקל, מגן מרוח, מתקפל לגודל המתאים לכיס.",2028d2a0-80b4-4284-949e-db6730a35c50,2013-04-30 00:00:00.000


In [119]:
#merging cat and description

description = pd.merge(cat, descr, how = 'inner', on = 'ProductModelID', suffixes=('_cat', '_descr')) #used suffixes because merge producing duplicate columns
description.head()

,ProductID,Name_x,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid_x,ModifiedDate_x_cat,ProductCategoryID,Name_y,rowguid_y,ModifiedDate_y_cat,Name,rowguid_cat,ModifiedDate,ProductDescriptionID,CultureID,ModifiedDate_x_descr,Description,rowguid_descr,ModifiedDate_y_descr
0,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.31,1431.5,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30,1090,en,2013-04-30,Our lightest and best quality aluminum frame m...,7054c83c-bd9b-4e24-8476-4d1b37bb08df,2013-04-30 00:00:00.000
1,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.31,1431.5,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30,1451,ar,2013-04-30,لقد تم صناعة هيكل دراجتنا الألومنيوم الأخف وزن...,f5026880-a5e7-48de-92b0-757a4bd25cc7,2013-04-30 00:00:00.000
2,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.31,1431.5,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30,1573,fr,2013-04-30,Notre cadre en aluminium plus léger et de qual...,b428a17d-8a1f-48e2-9a79-cf045bdf775e,2013-04-30 00:00:00.000
3,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.31,1431.5,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30,1696,th,2013-04-30,เฟรมอลูมิเนียมคุณภาพสูงสุดและน้ำหนักเบาที่สุด ...,ce0f9929-1010-44d8-a4d3-308885fa3958,2014-02-08 10:32:17.973
4,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.31,1431.5,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43dd68d6-14a4-461f-9069-55309d90ea7e,2014-02-08 10:01:36.827,2,Road Frames,5515f857-075b-4f9a-87b7-43b4997077b3,2008-04-30,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30,1827,he,2013-04-30,מסגרת האלומיניום הקלה והאיכותית ביותר שלנו עשו...,cd2f31a9-b430-4d24-9cab-5a9294033d0d,2013-04-30 00:00:00.000


In [120]:
description[['ProductCategoryID', 'Name','ProductSubcategoryID', 'Name_x', 'Description']].head(15)

,ProductCategoryID,Name,ProductSubcategoryID,Name_x,Description
0,2,Components,14.0,"HL Road Frame - Black, 58",Our lightest and best quality aluminum frame m...
1,2,Components,14.0,"HL Road Frame - Black, 58",لقد تم صناعة هيكل دراجتنا الألومنيوم الأخف وزن...
2,2,Components,14.0,"HL Road Frame - Black, 58",Notre cadre en aluminium plus léger et de qual...
3,2,Components,14.0,"HL Road Frame - Black, 58",เฟรมอลูมิเนียมคุณภาพสูงสุดและน้ำหนักเบาที่สุด ...
4,2,Components,14.0,"HL Road Frame - Black, 58",מסגרת האלומיניום הקלה והאיכותית ביותר שלנו עשו...
5,2,Components,14.0,"HL Road Frame - Black, 58",重量最轻、质量最好的铝制车架采用最新的合金材质；经过焊接和热处理，坚固牢靠。我们创新的设计确...
6,2,Components,14.0,"HL Road Frame - Red, 58",Our lightest and best quality aluminum frame m...
7,2,Components,14.0,"HL Road Frame - Red, 58",لقد تم صناعة هيكل دراجتنا الألومنيوم الأخف وزن...
8,2,Components,14.0,"HL Road Frame - Red, 58",Notre cadre en aluminium plus léger et de qual...
9,2,Components,14.0,"HL Road Frame - Red, 58",เฟรมอลูมิเนียมคุณภาพสูงสุดและน้ำหนักเบาที่สุด ...


In [121]:
# 39 List the name, weight, and size of all products

prod = pd.read_sql_table(schema = 'Production', table_name = 'Product', con = engine)
prod.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [122]:
prod[['Name', 'Weight', 'Size']].tail(15)

,Name,Weight,Size
491,"Mountain-500 Silver, 48",28.42,48
492,"Mountain-500 Silver, 52",28.68,52
493,"Mountain-500 Black, 40",27.35,40
494,"Mountain-500 Black, 42",27.77,42
495,"Mountain-500 Black, 44",28.13,44
496,"Mountain-500 Black, 48",28.42,48
497,"Mountain-500 Black, 52",28.68,52
498,LL Bottom Bracket,223.00,None
499,ML Bottom Bracket,168.00,None
500,HL Bottom Bracket,170.00,None


In [123]:
# 40 Display the order date, customer ID, and total due amount for all orders

soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [124]:
soh[['SalesOrderID', 'OrderDate', 'CustomerID', 'TotalDue']].head(15)

,SalesOrderID,OrderDate,CustomerID,TotalDue
0,43660,2011-05-20,29672,1457.3288
1,43661,2011-05-31,29734,36865.8012
2,43662,2011-05-31,29994,32474.9324
3,43663,2011-05-31,29565,472.3108
4,43664,2011-05-31,29898,27510.4109
5,43665,2011-05-31,29580,16158.6961
6,43666,2011-05-31,30052,5694.8564
7,43667,2011-05-31,29974,6876.3649
8,43668,2011-05-31,29614,40487.7233
9,43669,2011-05-31,29747,807.2585


In [125]:
# 41 Retrieve all columns for the Product table

prod = pd.read_sql_table(schema = 'Production', table_name = 'Product', con = engine)
prod.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [126]:
# 42 Get all columns for the Employee table

query = '''
SELECT 
    BusinessEntityID,
    NationalIDNumber,
    LoginID,
    CONVERT(varchar(max), OrganizationNode) AS OrganizationNode, 
    OrganizationLevel,
    JobTitle,
    BirthDate,
    MaritalStatus,
    Gender,
    HireDate,
    SalariedFlag,
    VacationHours,
    SickLeaveHours,
    CurrentFlag,
    rowguid,
    ModifiedDate
FROM 
    HumanResources.Employee'''
emp = pd.read_sql_query(query, con = engine)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,None,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,/1/,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,/1/1/,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,/1/1/1/,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,/1/1/2/,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [127]:
#43 Retrieve all columns for the SalesOrderHeader table

soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con = engine)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07


In [128]:
#44 Fetch all columns for the Customer table

cus = pd.read_sql_table(schema = 'Sales', table_name = 'Customer', con = engine)
cus.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [129]:
# 45 Get all columns for the ProductCategory table

cat = pd.read_sql_table(schema = 'Production', table_name = 'ProductCategory', con = engine)
cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30


In [130]:
# 46 Insert a new product with the specified name, list price, and color

prod = pd.read_sql_table(schema = 'Production', table_name = 'Product', con = engine)
prod.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


In [131]:
new = pd.DataFrame({'Name' : ['New_Product'], 'Color' : ['New_Color'], 'ListPrice' : [0]})
new

,Name,Color,ListPrice
0,New_Product,New_Color,0


In [132]:
pd.concat([prod, new]).tail(10)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
497,993.0,"Mountain-500 Black, 52",BK-M18B-52,True,True,Black,100.0,75.0,294.5797,146.674,52,CM,LB,28.68,4.0,M,L,U,1.0,23.0,2013-05-30 00:00:00,NaT,NaT,69ee3b55-e142-4e4f-aed8-af02978fbe87,2014-02-08 10:01:36.827
498,994.0,LL Bottom Bracket,BB-7421,True,True,None,500.0,375.0,23.9716,53.990,None,None,G,223.00,1.0,None,L,None,5.0,95.0,2013-05-30 00:00:00,NaT,NaT,fa3c65cd-0a22-47e3-bdf6-53f1dc138c43,2014-02-08 10:01:36.827
499,995.0,ML Bottom Bracket,BB-8107,True,True,None,500.0,375.0,44.9506,101.240,None,None,G,168.00,1.0,None,M,None,5.0,96.0,2013-05-30 00:00:00,NaT,NaT,71ab847f-d091-42d6-b735-7b0c2d82fc84,2014-02-08 10:01:36.827
500,996.0,HL Bottom Bracket,BB-9108,True,True,None,500.0,375.0,53.9416,121.490,None,None,G,170.00,1.0,None,H,None,5.0,97.0,2013-05-30 00:00:00,NaT,NaT,230c47c5-08b2-4ce3-b706-69c0bdd62965,2014-02-08 10:01:36.827
501,997.0,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100.0,75.0,343.6496,146.674,44,CM,LB,19.77,4.0,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,44ce4802-409f-43ab-9b27-ca53421805be,2014-02-08 10:01:36.827
502,998.0,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100.0,75.0,343.6496,146.674,48,CM,LB,20.13,4.0,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,3de9a212-1d49-40b6-b10a-f564d981dbde,2014-02-08 10:01:36.827
503,999.0,"Road-750 Black, 52",BK-R19B-52,True,True,Black,100.0,75.0,343.6496,146.674,52,CM,LB,20.42,4.0,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,ae638923-2b67-4679-b90e-abbab17dca31,2014-02-08 10:01:36.827
504,1001.0,Vintage Handcrafted Wooden Handlebars,VHWH1000,True,True,GOLD,3.0,4.0,6.9223,8.900,M,CM,LB,1.50,1.0,M,H,U,23.0,18.0,2011-05-31 10:00:00,2011-08-31 10:00:00,2012-05-31 10:00:00,4fbd47f2-b7e5-48b0-928e-45b8933a56da,2014-02-08 10:02:36.827
505,2006.0,BIKE HANDLE,RV-H123,True,True,BLUE,250.0,20.0,145.8800,2000.000,20,CM,LB,2.00,0.0,None,None,None,NaN,NaN,2008-04-30 00:00:00,NaT,NaT,d67538f2-a81a-4a4f-98f7-202002d888b4,2024-03-02 14:57:55.763
0,NaN,New_Product,NaN,NaN,NaN,New_Color,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaT


In [133]:
#47 Add a new employee with the given first name, last name, and email

person = pd.read_sql_table(schema = 'Person', table_name = 'Person', con = engine)
person.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92c4279f-1207-48a3-8448-4636514eb7e2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",d8763459-8aa8-47cc-aff7-c9079af79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",e1a2555e-0828-434b-a33b-6f38136a37de,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f2d7ce06-38b3-4357-805b-f4b6b71c01ff,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",f3a3f6b4-ae3b-430c-a754-9f2231ba6fef,2007-12-30


In [134]:
email = pd.read_sql_table(schema = 'Person', table_name = 'EmailAddress', con = engine)
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8a1901e4-671b-431a-871c-eadb2942e9ee,2009-01-07
1,2,2,terri0@adventure-works.com,b5ff9efd-72a2-4f87-830b-f338fdd4d162,2008-01-24
2,3,3,roberto0@adventure-works.com,c8a51084-1c03-4c58-a8b3-55854ae7c499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ed1-0031-4b4a-afd2-77487a556b3b,2007-11-28
4,5,5,gail0@adventure-works.com,e76d2ea3-08e5-409c-bbe2-5dd1cdf89a3b,2007-12-30


In [135]:
per = pd.DataFrame({'BusinessEntityID' : [12345], 'PersonType' : ['EM'], 'FirstName' : ['Newone'], 'LastName' : ['New']})
per

,BusinessEntityID,PersonType,FirstName,LastName
0,12345,EM,Newone,New


In [136]:
pd.concat([person, per], ignore_index =True).tail()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
19969,20775,IN,False,None,Crystal,S,He,None,0.0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",cd0b82d0-3409-4497-a65f-a68e75ee26cb,2014-03-12
19970,20776,IN,False,None,Crystal,None,Zheng,None,1.0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",386e35e8-97b3-43e4-9b5f-9594c2ff5cea,2014-01-14
19971,20777,IN,False,None,Crystal,None,Hu,None,0.0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",1773c8f3-2cc7-4826-a7f5-30d3f36543cd,2013-10-16
19972,20778,EM,False,None,ARAVINDH,None,A,None,0.0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",5d441932-109a-4312-80f9-5aa0214728a1,2009-01-22
19973,12345,EM,NaN,NaN,Newone,NaN,New,NaN,NaN,NaN,NaN,NaN,NaT


In [137]:
mail = pd.DataFrame({'BusinessEntityID' : [12345], 'EmailAddressID' : [12345], 'EmailAddress' : ['newone@nomail,com']})
mail

,BusinessEntityID,EmailAddressID,EmailAddress
0,12345,12345,"newone@nomail,com"


In [138]:
pd.concat([email, mail], ignore_index = True).tail()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
19969,20775,19970,crystal19@adventure-works.com,f3114a36-b573-4cf4-a212-6ac42261dfce,2014-03-12
19970,20776,19971,crystal20@adventure-works.com,287006e5-4cb1-4688-9a2f-479f048f27ce,2014-01-14
19971,20777,19972,crystal21@adventure-works.com,012bbebe-fc99-4c4e-b2bf-6f21e2cf19d3,2013-10-16
19972,20778,19973,aravndh@hotmail.com,052b671a-1fab-4fe2-b45c-035882dfcf34,2009-01-22
19973,12345,12345,"newone@nomail,com",NaN,NaT


In [139]:
# 48 Insert a new customer with the provided details

cus = pd.read_sql_table(schema = 'Sales', table_name = 'Customer', con = engine)
cus.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1a92df88-bfa2-467d-bd54-fcb9e647fdd7,2014-09-12 11:15:07.263


In [140]:
new_cus = pd.DataFrame({'CustomerID' : [12345], 'PersonID' : [12345], 'TerritoryID' : [1]})
new_cus

,CustomerID,PersonID,TerritoryID
0,12345,12345,1


In [141]:
pd.concat([cus, new_cus], ignore_index = True).tail()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
19817,30117,1991.0,1992.0,4,AW00030117,6f08e2fb-1cd3-4f6e-a2e6-385669598b19,2014-09-12 11:15:07.263
19818,30118,1993.0,1994.0,3,AW00030118,2495b4eb-fe8b-459e-a1b6-dba25c04e626,2014-09-12 11:15:07.263
19819,30124,NaN,1956.0,10,AW00030124,7a4115b1-6f02-451a-9ae0-1de1cccaebd7,2014-09-12 11:15:07.263
19820,30125,NaN,1956.0,10,AW00030125,4a0428e9-1b82-49df-b7d7-6969b4db07e5,2014-09-12 11:15:07.263
19821,12345,12345.0,NaN,1,NaN,NaN,NaT


In [142]:
# 49 Add a new record into the ProductCategory table

pc = pd.read_sql_table(schema = 'Production', table_name = 'ProductCategory', con =engine)
pc

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30


In [143]:
new_pc = pd.DataFrame({'ProductCategoryID' : [5], 'Name' : ['Want to add']})
new_pc

,ProductCategoryID,Name
0,5,Want to add


In [144]:
pd.concat([pc, new_pc], ignore_index= True)

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30
1,2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30
2,3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30
3,4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30
4,5,Want to add,NaN,NaT


In [145]:
# 50 Insert a new record into the SalesOrderHeader table

soh = pd.read_sql_table(schema = 'Sales', table_name = 'SalesOrderHeader', con =engine)
soh.tail()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
31460,75120,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75120,None,10-4030-018749,18749,NaN,6,28374,28374,1,8925.0,929849Vi46003,NaN,84.9600,6.7968,2.1240,93.8808,None,ae6a4fcf-ff73-4cd4-af2c-5993d00d4afe,2014-07-07
31461,75121,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75121,None,10-4030-015251,15251,NaN,6,26553,26553,1,14220.0,529864Vi73738,NaN,74.9800,5.9984,1.8745,82.8529,None,d7395c0e-00cb-4bfa-a238-0d6a9f49884f,2014-07-07
31462,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,15868,NaN,6,14616,14616,1,18719.0,330022Vi97312,NaN,30.9700,2.4776,0.7743,34.2219,None,4221035a-4159-492f-af40-4363a64ffc16,2014-07-07
31463,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,18759,NaN,6,14024,14024,1,10084.0,230370Vi51970,NaN,189.9700,15.1976,4.7493,209.9169,None,d54752ff-2b54-4be5-95ea-3b72289c059f,2014-07-07
31464,75124,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO75124,PO18009186470,-4656,29614,283.0,5,517,849,5,15232.0,55680Vi53503,4.0,20565.6206,1971.5149,616.0984,23153.2339,None,0f91dbcb-04ad-4d4d-aeed-c2af783d81b9,2011-06-07


In [146]:
new = pd.DataFrame({'SalesOrderID' : [8000], 'OrderDate' :[pd.Timestamp.today()], 'DueDate' : [pd.Timestamp.today() + pd.Timedelta(days =3 )], 'ShipDate' : [pd.Timestamp.today() + pd.Timedelta(days = 4)], 'CustomerID' : [12345]})
new

,SalesOrderID,OrderDate,DueDate,ShipDate,CustomerID
0,8000,2024-06-05 18:06:22.626894,2024-06-08 18:06:22.627401,2024-06-09 18:06:22.629170,12345


In [147]:
pd.concat([soh, new], ignore_index = True).tail()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
31461,75121,8.0,2014-06-30 00:00:00.000000,2014-07-12 00:00:00.000000,2014-07-07 00:00:00.000000,5.0,True,SO75121,None,10-4030-015251,15251,NaN,6.0,26553.0,26553.0,1.0,14220.0,529864Vi73738,NaN,74.9800,5.9984,1.8745,82.8529,None,d7395c0e-00cb-4bfa-a238-0d6a9f49884f,2014-07-07
31462,75122,8.0,2014-06-30 00:00:00.000000,2014-07-12 00:00:00.000000,2014-07-07 00:00:00.000000,5.0,True,SO75122,None,10-4030-015868,15868,NaN,6.0,14616.0,14616.0,1.0,18719.0,330022Vi97312,NaN,30.9700,2.4776,0.7743,34.2219,None,4221035a-4159-492f-af40-4363a64ffc16,2014-07-07
31463,75123,8.0,2014-06-30 00:00:00.000000,2014-07-12 00:00:00.000000,2014-07-07 00:00:00.000000,5.0,True,SO75123,None,10-4030-018759,18759,NaN,6.0,14024.0,14024.0,1.0,10084.0,230370Vi51970,NaN,189.9700,15.1976,4.7493,209.9169,None,d54752ff-2b54-4be5-95ea-3b72289c059f,2014-07-07
31464,75124,8.0,2011-05-31 00:00:00.000000,2011-06-12 00:00:00.000000,2011-06-07 00:00:00.000000,5.0,False,SO75124,PO18009186470,-4656,29614,283.0,5.0,517.0,849.0,5.0,15232.0,55680Vi53503,4.0,20565.6206,1971.5149,616.0984,23153.2339,None,0f91dbcb-04ad-4d4d-aeed-c2af783d81b9,2011-06-07
31465,8000,NaN,2024-06-05 18:06:22.626894,2024-06-08 18:06:22.627401,2024-06-09 18:06:22.629170,NaN,NaN,NaN,NaN,NaN,12345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


: 